# Image Folder

`ImageFolder`를 사용해 실제 데이터를 불러오고, 모델을 학습해보자.

## 1. 데이터셋 분리

In [1]:
import os

dataset_dir = "../datasets/hands_data/"

train_dir = os.path.join(dataset_dir, "train_data")
if not os.path.isdir(train_dir):
    os.mkdir(train_dir)
    
train_zero_dir = os.path.join(train_dir, "0_front")
if not os.path.isdir(train_zero_dir):
    os.mkdir(train_zero_dir)
    
train_five_dir = os.path.join(train_dir, "5_front")
if not os.path.isdir(train_five_dir):
    os.mkdir(train_five_dir)
    
test_dir = os.path.join(dataset_dir, "test_data")
if not os.path.isdir(test_dir):
    os.mkdir(test_dir)
    
test_zero_dir = os.path.join(test_dir, "0_front")
if not os.path.isdir(test_zero_dir):
    os.mkdir(test_zero_dir)
    
test_five_dir = os.path.join(test_dir, "5_front")
if not os.path.isdir(test_five_dir):
    os.mkdir(test_five_dir)

In [2]:
zero_front_imgs = os.listdir(os.path.join(dataset_dir, "0_front"))
five_front_imgs = os.listdir(os.path.join(dataset_dir, "5_front"))

num_test_zero_front = len(zero_front_imgs)//3
test_zero_front_imgs = zero_front_imgs[:num_test_zero_front]
train_zero_front_imgs = zero_front_imgs[num_test_zero_front:]

num_test_five_front = len(five_front_imgs)//3
test_five_front_imgs = five_front_imgs[:num_test_five_front]
train_five_front_imgs = five_front_imgs[num_test_five_front:]

In [3]:
import shutil

for img in train_zero_front_imgs:
    shutil.copy(os.path.join(dataset_dir, "0_front", img),
                os.path.join(train_zero_dir, img))
    
for img in test_zero_front_imgs:
    shutil.copy(os.path.join(dataset_dir, "0_front", img),
                os.path.join(test_zero_dir, img))
    
for img in train_five_front_imgs:
    shutil.copy(os.path.join(dataset_dir, "5_front", img),
                os.path.join(train_five_dir, img))
    
for img in test_five_front_imgs:
    shutil.copy(os.path.join(dataset_dir, "5_front", img),
                os.path.join(test_five_dir, img))    

## 2. `ImageFolder`를 사용해 `DataLoader`로 변환

In [4]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [6]:
trans = transforms.Compose([
    transforms.Resize((130, 100)), # resize
    transforms.ToTensor() # normalize (0~255 -> 0~1)
])

train_data = torchvision.datasets.ImageFolder(root=train_dir,
                                              transform=trans)

train_loader = DataLoader(train_data,
                          batch_size=24,
                          shuffle=True,
                          num_workers=2)

## 3. 학습

In [7]:
from torch import nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.linear1 = nn.Linear(64*6*4, 128, bias=True)
        self.linear2 = nn.Linear(128, 128, bias=True)
        self.linear3 = nn.Linear(128, 2, bias=True)
        
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)
        nn.init.xavier_normal_(self.linear3.weight)
        
        self.fc = nn.Sequential(
            self.linear1,
            nn.ReLU(),
            self.linear2,
            nn.ReLU(),
            self.linear3
        )
        
    def forward(self, x):
        x = self.conv1(x)        
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x


model = CNN().to(device)

In [8]:
from torch import optim

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
total_batch = len(train_loader)

epochs = 30
for epoch in range(epochs):
    avg_loss = 0.0
    
    for num, data in enumerate(train_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_loss += loss / total_batch
        
    print('[Epoch:{}] loss = {}'.format(epoch+1, avg_loss))
    
print('Learning Finished!')

[Epoch:1] loss = 0.7008646130561829
[Epoch:2] loss = 0.6930096745491028
[Epoch:3] loss = 0.6972765922546387
[Epoch:4] loss = 0.6918877959251404
[Epoch:5] loss = 0.6682376265525818
[Epoch:6] loss = 0.4974442422389984
[Epoch:7] loss = 0.2836402654647827
[Epoch:8] loss = 0.17113745212554932
[Epoch:9] loss = 0.1776367872953415
[Epoch:10] loss = 0.09619373828172684
[Epoch:11] loss = 0.062125932425260544
[Epoch:12] loss = 0.037254080176353455
[Epoch:13] loss = 0.030431758612394333
[Epoch:14] loss = 0.022591782733798027
[Epoch:15] loss = 0.011807529255747795
[Epoch:16] loss = 0.01299683004617691
[Epoch:17] loss = 0.0371953509747982
[Epoch:18] loss = 0.018985679373145103
[Epoch:19] loss = 0.004967097193002701
[Epoch:20] loss = 0.0026736478321254253
[Epoch:21] loss = 0.0010229445761069655
[Epoch:22] loss = 0.0007375929271802306
[Epoch:23] loss = 0.0005534369847737253
[Epoch:24] loss = 0.00043675527558662
[Epoch:25] loss = 0.00035433401353657246
[Epoch:26] loss = 0.00029248613282106817
[Epoch:27

## 4. 모델 저장 및 불러오기

In [10]:
torch.save(model.state_dict(), "../models/lab_10_4_model.pth")

In [11]:
loaded_model = CNN().to(device)
loaded_model.load_state_dict(torch.load("../models/lab_10_4_model.pth"))

<All keys matched successfully>

## 5. 모델 평가

In [12]:
test_data = torchvision.datasets.ImageFolder(root=test_dir,
                                             transform=trans)

test_loader = DataLoader(test_data, batch_size=len(test_data))

In [13]:
with torch.no_grad():
    for num, data in enumerate(test_loader):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = loaded_model(imgs)
        correct_prediction = torch.argmax(prediction, 1) == label
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 0.9878787398338318
